<a href="https://colab.research.google.com/github/kuaminika/investingNotes/blob/master/Copy_of_options_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoo_fin &> /dev/null
!pip install yfinance &> /dev/null
import pandas as pd
import numpy as np
import yfinance as yf
import yahoo_fin.options as oi
from datetime import datetime as dt, time, timedelta
from dateutil.relativedelta import relativedelta
from pytz import timezone
from collections import defaultdict
from google.colab import data_table
import time as t


In [ ]:
#@title Parameters
ticker = "AMD INTC MU" #@param {type:"string"}
months_out = "3" #@param ["3", "6", "12", "24"]
maxBudget = "4.00" #@param {type:"string"}



In [ ]:
#**********************************************************************
# FUNCTIONS / METHODS
#**********************************************************************

def closestWeekday(x):
  if x.weekday() in set ((5,6)):
    vDate = x.replace(tzinfo=None)
    offset = max(1, (vDate.weekday() + 6) % 7 - 3)
    vTimedelta = timedelta(offset)
    most_recent = vDate - vTimedelta
  else:
    most_recent = x
  return most_recent

def nearest(items, compDate):
    return min(items, key=lambda x: abs(  x.date() - compDate.date() )) # ( x.strftime("%Y-%m-%d") () )or
    #return min(items, key=lambda x: abs(x - compDate))

def nearest_value(items, comp):
    return min(items, key=lambda x: abs(  x - comp ))

def total_keys(test_dict):
    return (0 if not isinstance(test_dict, dict) 
    else len(test_dict) + sum(total_keys(val) for val in test_dict.values()))

#**********************************************************************
# 
#**********************************************************************

#stock_data = yf.Ticker(ticker)
# get currnt date, delta date
currentDate = closestWeekday(dt.now(timezone('US/Eastern')))
monthDate = closestWeekday(currentDate + relativedelta(months=int(months_out)))
# fetch future date
futureDate = closestWeekday(currentDate + relativedelta(years=2))
#df = pd.DataFrame()
#df = stock_data.history(start=start_date, end=end_date, auto_adjust=True)
#df1 = stock_data.history(start=deltaDate, end=end_date, auto_adjust=True)
tickerString = str(ticker.replace(',',''))
tickerList = tickerString.split()

stockDatesArrList = [[]] * len(tickerList)
stockDatesDict = {}

# 1. create dictionary with lists of dates
print('------')

counter = 0
for x in tickerList:
  # fetch expiration dates
  #stockDatesArrList[counter] = oi.get_expiration_dates(x)
  stock_dates = oi.get_expiration_dates(x)
  # in order to avoid getting issues with the nearest date function, convert the dates to datetime format
  # TypeError: unsupported operand type(s) for -: 'str' and 'dt'
  stock_dl = [dt.strptime(stockDates, "%B %d, %Y") for stockDates in stock_dates]
  stockDatesDict.update( {tickerList[counter] : stock_dl } )
  counter += 1
  
longest = 0
#total = total_keys(stockDatesDict)

# find which set has the most expiry dates
# store the biggest number in variable longest
for key in stockDatesDict:
    print("{} has {} expiry dates".format(key,len(stockDatesDict[key])))
    for l in range(len(stockDatesDict[key])):
      if l > longest:
          longest = l
      l+=1

print('------')
print("stockDatesDict")
print(stockDatesDict)
print('------')

# 2. Create Stock Date DF while providing index
stockDateDF = pd.DataFrame() #
stockDateDF = stockDateDF.append(stockDatesDict, ignore_index=True)

#stockDateDF = pd.DataFrame(index=tickerList) 

#for e in range(len(tickerList)):
#  stockDateDF[tickerList[e]] = list(tickerArrList[e]) 
print('------')
print("stockDateDF")
data_table.DataTable(stockDateDF)
print('------')

# 3. find nearest expiry date in list of dates available
# iterate through dictionnary by looping through keys
# store keys in an array
dateToScanDict = {}
cntr = 0
for k in tickerList:
  dateToScanDict.update( {tickerList[cntr] : nearest(stockDatesDict[k],closestWeekday(monthDate).replace(tzinfo=None)) } )
  #print(nearest(stockDatesDict[k],closestWeekday(monthDate).replace(tzinfo=None)))
  cntr += 1

print("dateToScanDict")
print(dateToScanDict)

#print(oi.get_calls(ticker='MU',date='2022-02-25'))

providedList = []


        # Index(['Contract Name', 'Last Trade Date', 'Strike', 'Last Price', 'Bid', 'Ask', 'Change', '% Change', 'Volume', 'Open Interest', 'Implied Volatility'], dtype='object')
        #print(k)
        #print(startingIndex)
        #print(stockDateDF[k][0][startingIndex])
        #print(optionDF) #[["Contract Name","Strike","Last Price","Ask","Bid"]]
        #data_table.DataTable(df1)
        #vIndex = df1[df1["Bid"]==nearest_price].index.values
        #print("nearest price for {} on expiry date {}: {} @ position {}".format(k,nearest_date,nearest_price,vIndex))
        #vIndex1 = vIndex-2        
        #df_option1 = df1.iloc[vIndex[0]:vIndex1[0]][["Contract Name","Strike","Last Price","Ask","Bid"]]
        #print(df_option1)
print('-----------')


------
AMD has 15 expiry dates
INTC has 14 expiry dates
MU has 16 expiry dates
------
stockDatesDict
{'AMD': [datetime.datetime(2022, 1, 28, 0, 0), datetime.datetime(2022, 2, 4, 0, 0), datetime.datetime(2022, 2, 11, 0, 0), datetime.datetime(2022, 2, 18, 0, 0), datetime.datetime(2022, 2, 25, 0, 0), datetime.datetime(2022, 3, 4, 0, 0), datetime.datetime(2022, 3, 18, 0, 0), datetime.datetime(2022, 4, 14, 0, 0), datetime.datetime(2022, 5, 20, 0, 0), datetime.datetime(2022, 6, 17, 0, 0), datetime.datetime(2022, 7, 15, 0, 0), datetime.datetime(2022, 9, 16, 0, 0), datetime.datetime(2023, 1, 20, 0, 0), datetime.datetime(2023, 4, 21, 0, 0), datetime.datetime(2024, 1, 19, 0, 0)], 'INTC': [datetime.datetime(2022, 1, 28, 0, 0), datetime.datetime(2022, 2, 4, 0, 0), datetime.datetime(2022, 2, 11, 0, 0), datetime.datetime(2022, 2, 18, 0, 0), datetime.datetime(2022, 2, 25, 0, 0), datetime.datetime(2022, 3, 4, 0, 0), datetime.datetime(2022, 3, 18, 0, 0), datetime.datetime(2022, 4, 14, 0, 0), datetime.d

In [ ]:
ListOfPossibilities = list( {} for i in range(len(tickerList)) )

expiryDate   = []
contractName = []
strikePrice  = []
lastPrice    = []
contractBid  = []
contractAsk  = []

# 4.  for each ticker...
# 4.1 find index location for date in in dateToScanDict
# 4.1 iterate through every date starting from what we have in dateToScanDict
# 4.2 get option data based on each of those dates
# 4.3 find nearest value based on budget
# 4.4 store it inside of list of possibilities

#for t in tickerList, range(len(tickerList)):
for i, t in enumerate(tickerList):
    # assign index to variable in oder to iterate through remaining dates
    #print((stockDateDF[k][0]).index(dateToScanDict[k]))
    startingIndex = (stockDateDF[t][0]).index(dateToScanDict[t])
    #print("+++++++++")
    #print(startingIndex)
    #print(stockDateDF[t][0][startingIndex])
    #print("+++++++++")
    #for ( i = startingIndex; i < longest ; i ++)
    #for ( i = 0 ; i < longest ; i++ )
    for idx in range(startingIndex, (longest+1)): 
      try:
        nearest_date = stockDateDF[t][0][idx]
        #print(t)
        #print(nearest_date)
        providedList = oi.get_calls(t, nearest_date)[["Contract Name","Strike","Last Price","Bid","Ask"]]
        #print(type(providedList["Ask"]))
        nearest_price = nearest_value(providedList["Ask"],float(maxBudget))
        #print(nearest_date)
        nearestPriceIndex = providedList["Ask"][providedList["Ask"]==nearest_price].index[0]
        print("based on the index of the nearest price: {}".format(providedList.iloc[nearestPriceIndex]["Ask"]))
        print("index from price: {}".format(nearestPriceIndex))
        print("index from date:  {}".format(idx))
         
        expiryDate.append(nearest_date)
        contractName.append(providedList.iloc[nearestPriceIndex]["Contract Name"])
        strikePrice.append(providedList.iloc[nearestPriceIndex]["Strike"])
        lastPrice.append(providedList.iloc[nearestPriceIndex]["Last Price"])
        contractBid.append(providedList.iloc[nearestPriceIndex]["Bid"])
        contractAsk.append(providedList.iloc[nearestPriceIndex]["Ask"]) 

        # issue comes from iterating for the amount of dates starting from date index whereas we start at the index of the nearest price
        #t.sleep(2)
      except IndexError:
        pass
      #create dictionary which will hold the following:
      data = {
         "expiryDates" : expiryDate,
         "Contract Name" : contractName ,
         "Strike" : strikePrice ,
         "Last Price" : lastPrice  ,
         "Bid"        : contractBid  ,
         "Ask"        : contractAsk
       }
      #print(type(data))
      ListOfPossibilities[i] = dict(data)
      #print('-----------')
      #print(ListOfPossibilities[i])
      #print('-----------')
print(ListOfPossibilities)

+++++++++
7
2022-04-14 00:00:00
+++++++++
AMD
2022-04-14 00:00:00
based on the index of the nearest price: 4.35
index from price: 20
index from date:  7
AMD
2022-05-20 00:00:00
based on the index of the nearest price: 4.15
index from price: 17
index from date:  8
AMD
2022-06-17 00:00:00
based on the index of the nearest price: 3.9
index from price: 36
index from date:  9
AMD
2022-07-15 00:00:00
based on the index of the nearest price: 4.2
index from price: 20
index from date:  10
AMD
2022-09-16 00:00:00
based on the index of the nearest price: 4.1
index from price: 37
index from date:  11
AMD
2023-01-20 00:00:00
based on the index of the nearest price: 3.85
index from price: 46
index from date:  12
AMD
2023-04-21 00:00:00
based on the index of the nearest price: 5.75
index from price: 34
index from date:  13
AMD
2024-01-19 00:00:00
based on the index of the nearest price: 6.75
index from price: 42
index from date:  14
oops
+++++++++
7
2022-04-14 00:00:00
+++++++++
INTC
2022-04-14 00:00